In [1]:
#logistic regression

In [1]:
import pandas as pd
import numpy as np
from sklearn.decomposition import PCA
import matplotlib.pyplot as plt
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.neighbors import NearestNeighbors
from sklearn import neighbors
from sklearn.pipeline import make_pipeline
from sklearn.model_selection import GroupKFold
from sklearn.multiclass import OneVsOneClassifier
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import cross_val_predict
from sklearn import metrics
from sklearn.svm import SVC
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import LogisticRegressionCV
from sklearn.linear_model import LogisticRegression

In [2]:
data = pd.read_csv('C:/Users/zhanghui/Desktop/PROJECT/Frogs_MFCCs.csv')
data.head(5)

,MFCCs_ 1,MFCCs_ 2,MFCCs_ 3,MFCCs_ 4,MFCCs_ 5,MFCCs_ 6,MFCCs_ 7,MFCCs_ 8,MFCCs_ 9,MFCCs_10,...,MFCCs_17,MFCCs_18,MFCCs_19,MFCCs_20,MFCCs_21,MFCCs_22,Family,Genus,Species,RecordID
0,1.0,0.152936,-0.105586,0.200722,0.317201,0.260764,0.100945,-0.150063,-0.171128,0.124676,...,-0.108351,-0.077623,-0.009568,0.057684,0.118680,0.014038,Leptodactylidae,Adenomera,AdenomeraAndre,1
1,1.0,0.171534,-0.098975,0.268425,0.338672,0.268353,0.060835,-0.222475,-0.207693,0.170883,...,-0.090974,-0.056510,-0.035303,0.020140,0.082263,0.029056,Leptodactylidae,Adenomera,AdenomeraAndre,1
2,1.0,0.152317,-0.082973,0.287128,0.276014,0.189867,0.008714,-0.242234,-0.219153,0.232538,...,-0.050691,-0.023590,-0.066722,-0.025083,0.099108,0.077162,Leptodactylidae,Adenomera,AdenomeraAndre,1
3,1.0,0.224392,0.118985,0.329432,0.372088,0.361005,0.015501,-0.194347,-0.098181,0.270375,...,-0.136009,-0.177037,-0.130498,-0.054766,-0.018691,0.023954,Leptodactylidae,Adenomera,AdenomeraAndre,1
4,1.0,0.087817,-0.068345,0.306967,0.330923,0.249144,0.006884,-0.265423,-0.172700,0.266434,...,-0.048885,-0.053074,-0.088550,-0.031346,0.108610,0.079244,Leptodactylidae,Adenomera,AdenomeraAndre,1


In [3]:
X_test = np.array(data.ix[:,:22])
y_test = np.array(data['Family'])
np.unique(y_test)

array(['Bufonidae', 'Dendrobatidae', 'Hylidae', 'Leptodactylidae'], dtype=object)

In [4]:
#crossvalidation
groups = np.array(data['RecordID'])
gkf = GroupKFold(n_splits=60)
gkf.split(X_test, y_test, groups)

#LR on anural
lr = LogisticRegressionCV()

tuned_parameters = {'Cs': [5, 10],
                     'solver': ['lbfgs','newton-cg','sag']}

clf = GridSearchCV(estimator=lr, 
                   param_grid=tuned_parameters, 
                   cv=list(gkf.split(X_test, 
                                     y_test, 
                                     groups)))

clf.fit(X_test, y_test)

predicted = cross_val_predict(clf.best_estimator_,
                              X_test, 
                              y_test, 
                              cv=list(gkf.split(X_test, 
                                            y_test, 
                                            groups)))


C:\Users\zhanghui\Anaconda3\lib\site-packages\sklearn\linear_model\sag.py:286: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  "the coef_ did not converge", ConvergenceWarning)
C:\Users\zhanghui\Anaconda3\lib\site-packages\sklearn\linear_model\sag.py:286: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  "the coef_ did not converge", ConvergenceWarning)
C:\Users\zhanghui\Anaconda3\lib\site-packages\sklearn\linear_model\sag.py:286: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  "the coef_ did not converge", ConvergenceWarning)
C:\Users\zhanghui\Anaconda3\lib\site-packages\sklearn\linear_model\sag.py:286: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  "the coef_ did not converge", ConvergenceWarning)
C:\Users\zhanghui\Anaconda3\lib\site-packages\sklearn\linear_model\sag.py:286: ConvergenceWarning: The max_iter was reached which means the 

In [8]:
index = list()
for i in range(len(predicted)):
    if predicted[i] == 'Leptodactylidae':
        index.append(i)
                
for i in range(4):
    print(clf.best_estimator_.predict_proba(X_test)[index,i].mean())
    
for i in range(4):
    print(clf.best_estimator_.decision_function(X_test)[index,i].mean())

0.00994773270333
0.0792413062928
0.0409135923165
0.869897368687
-4.65342864753
-2.51288036394
-4.25547266103
3.32628268507


In [486]:
#matrix on family level
result = np.c_[y_test, predicted]

Bufonidae = 0
Dendrobatidae = 0
Hylidae = 0
Leptodactylidae = 0
total = 0

for i in range(len(y_test)):
    if result[:, 0][i] == 'Hylidae':
        total += 1
        if result[:, 1][i] == 'Bufonidae':
            Bufonidae += 1
        if result[:, 1][i] == 'Dendrobatidae':
            Dendrobatidae += 1
        if result[:, 1][i] == 'Hylidae':
            Hylidae += 1
        if result[:, 1][i] == 'Leptodactylidae':
            Leptodactylidae += 1


Bufonidae, Dendrobatidae, Hylidae, Leptodactylidae, total

(0, 16, 1907, 242, 2165)

In [478]:
for i in range(4):
    print(lr.predict_proba(X_test)[:,i].mean())

0.0109487365237
0.0876132159065
0.316515336821
0.584922710748


In [22]:
data = np.array(data)
data.shape

(7195, 26)

In [64]:
X_Leptodactylidae = list()
for i in range(len(predicted)):
    if predicted[i] == 'Leptodactylidae':
        X_Leptodactylidae.append(data[i])        
X_Leptodactylidae = np.array(X_Leptodactylidae)
y_Leptodactylidae = np.array(X_Leptodactylidae[:,23])

X_Dendrobatidae = list()
for i in range(len(predicted)):
    if predicted[i] == 'Dendrobatidae':
        X_Dendrobatidae.append(data[i])        
X_Dendrobatidae = np.array(X_Dendrobatidae)
y_Dendrobatidae = np.array(X_Dendrobatidae[:,23])

X_Hylidae = list()
for i in range(len(predicted)):
    if predicted[i] == 'Hylidae':
        X_Hylidae.append(data[i])
X_Hylidae = np.array(X_Hylidae)
y_Hylidae = np.array(X_Hylidae[:,23])

X_Bufonidae = list()
for i in range(len(predicted)):
    if predicted[i] == 'Bufonidae':
        X_Bufonidae.append(data[i])        
X_Bufonidae = np.array(X_Bufonidae)
y_Bufonidae = np.array(X_Bufonidae[:,23])


In [436]:
#data prepare
print(np.unique(X_Bufonidae[:,23]))
X_Bufonidae_test = X_Bufonidae[:,:22]

#group prepare
print(np.unique(X_Bufonidae[:,25]).size)
groups_Bufonidae = np.array(X_Bufonidae[:,25])

['Adenomera' 'Ameerega' 'Dendropsophus' 'Hypsiboas' 'Leptodactylus'
 'Osteocephalus' 'Rhinella' 'Scinax']
10


In [437]:
#crossvalidation
gkf = GroupKFold(n_splits=10)
gkf.split(X_Bufonidae_test, y_Bufonidae, groups_Bufonidae)

#SVM on genus family
lr = LogisticRegressionCV(cv=list(gkf.split(X_Bufonidae_test, 
                                            y_Bufonidae, 
                                            groups_Bufonidae)))

#tuned_parameters = {'solver': ('newton-cg', 'lbfgs', 'sag')}

#clf = GridSearchCV(estimator=lr, 
#                   param_grid=tuned_parameters, 
#                   cv=list(gkf.split(X_Bufonidae_test, 
#                                     y_Bufonidae, 
#                                     groups_Bufonidae)))


lr.fit(X_Bufonidae_test, y_Bufonidae)

predicted_Bufonidae = lr.predict(X_Bufonidae_test)

In [439]:
index = list()
for i in range(len(predicted_Bufonidae)):
    if predicted_Bufonidae[i] == 'Rhinella':
        index.append(i)
                
for i in range(8):
    print(lr.predict_proba(X_Bufonidae_test)[index,i].mean())
    
for i in range(8):
    print(lr.decision_function(X_Bufonidae_test)[index,i].mean())

0.0831234911866
0.0554160830737
0.0831250757355
0.0025162460393
0.0277057761601
0.055415976351
0.609573266128
0.0831240853255
-2.48490950839
-2.91776555394
-2.48488908206
-7.21662955661
-3.63766498667
-2.91776754699
0.257826637736
-2.48490186106


In [73]:
for i in range(8):
    print(lr.predict_proba(X_Bufonidae_test)[:,i].mean())

0.0798527619662
0.0532353479982
0.079853524132
0.0417677139944
0.026615609096
0.0532354440952
0.585586552913
0.0798530458052


In [434]:
#data prepare
print(np.unique(X_Hylidae[:,23]))
X_Hylidae_test = X_Hylidae[:,:22]

#group prepare
print(np.unique(X_Hylidae[:,25]).size)
groups_Hylidae = np.array(X_Hylidae[:,25])

['Adenomera' 'Ameerega' 'Dendropsophus' 'Hypsiboas' 'Leptodactylus'
 'Osteocephalus' 'Rhinella' 'Scinax']
48


In [435]:
#crossvalidation
gkf = GroupKFold(n_splits=48)
gkf.split(X_Hylidae_test, y_Hylidae, groups_Hylidae)

#LR on anural
lr = LogisticRegressionCV(cv=list(gkf.split(X_Hylidae_test, 
                                     y_Hylidae, 
                                     groups_Hylidae)))

lr.fit(X_Hylidae_test, y_Hylidae)

predicted_Hylidae = lr.predict(X_Hylidae_test) 


KeyboardInterrupt: 

0.0277170340081
0.0169689832101
0.0855928189655
0.714810625735
0.0134180331457
0.0571365315378
0.0226256323778
0.0617303410205


In [378]:
for i in range(8):
    print(lr.predict_proba(X_Hylidae_test)[index,i].mean())

0.0254628626711
0.015602495256
0.804939903507
0.0628812851095
0.00493997798194
0.0523776751935
0.020782285586
0.013013514695


In [379]:
for i in range(8):
    print(lr.decision_function(X_Hylidae_test)[index,i].mean())

-3.7527706195
-4.25156731565
4.75356733982
-4.47524130662
-9.08281755074
-3.00639913872
-3.96009794739
-10.8391923895


In [430]:
#data prepare
print(np.unique(X_Dendrobatidae[:,23]))
X_Dendrobatidae_test = X_Dendrobatidae[:,:22]

#group prepare
print(np.unique(X_Dendrobatidae[:,25]).size)
groups_Dendrobatidae = np.array(X_Dendrobatidae[:,25])

['Adenomera' 'Ameerega' 'Dendropsophus' 'Leptodactylus' 'Osteocephalus']
20


In [431]:
#crossvalidation
gkf = GroupKFold(n_splits=20)
gkf.split(X_Dendrobatidae_test, y_Dendrobatidae, groups_Dendrobatidae)

#LR on anural
lr = LogisticRegressionCV(cv=list(gkf.split(X_Dendrobatidae_test, 
                                            y_Dendrobatidae, 
                                            groups_Dendrobatidae)))


lr.fit(X_Dendrobatidae_test, y_Dendrobatidae)

predicted_Dendrobatidae = lr.predict(X_Dendrobatidae_test)
#predicted_Leptodactylidae = cross_val_predict(lr, 
#                                        X_Dendrobatidae_test, 
#                                        y_Dendrobatidae, 
#                                        cv=list(gkf.split(X_Dendrobatidae_test, 
#                                                          y_Dendrobatidae, 
#                                                          groups_Dendrobatidae)))

In [433]:
index = list()
for i in range(len(predicted_Dendrobatidae)):
    if predicted_Dendrobatidae[i] == 'Ameerega':
        index.append(i)
                
for i in range(5):
    print(lr.predict_proba(X_Dendrobatidae_test)[index,i].mean())
    
for i in range(5):
    print(lr.decision_function(X_Dendrobatidae_test)[index,i].mean())

0.0681002057439
0.89067911623
0.0376343250845
0.0017945247914
0.00179182815026
-2.61624270633
2.0977171029
-3.24147566776
-6.32121667484
-6.32272321047


In [86]:
for i in range(5):
    print(lr.predict_proba(X_Dendrobatidae_test)[:,i].mean())

0.0681002057439
0.89067911623
0.0376343250845
0.0017945247914
0.00179182815026


In [426]:
#data prepare
print(np.unique(X_Leptodactylidae[:,23]))
X_Leptodactylidae_test = X_Leptodactylidae[:,:22]

#group prepare
print(np.unique(X_Leptodactylidae[:,25]).size)
groups_Leptodactylidae = np.array(X_Leptodactylidae[:,25])

['Adenomera' 'Ameerega' 'Dendropsophus' 'Hypsiboas' 'Leptodactylus'
 'Osteocephalus' 'Rhinella' 'Scinax']
46


In [427]:
#crossvalidation
gkf = GroupKFold(n_splits=46)
gkf.split(X_Leptodactylidae_test, y_Leptodactylidae, groups_Leptodactylidae)

#LR on anural
lr = LogisticRegressionCV(cv=list(gkf.split(X_Leptodactylidae_test, 
                                            y_Leptodactylidae, 
                                            groups_Leptodactylidae)))


lr.fit(X_Leptodactylidae_test, y_Leptodactylidae)

predicted_Leptodactylidae = lr.predict(X_Leptodactylidae_test)

In [429]:
index = list()
for i in range(len(predicted_Leptodactylidae)):
    if predicted_Leptodactylidae[i] == 'Adenomera':
        index.append(i)
                
for i in range(8):
    print(lr.predict_proba(X_Leptodactylidae_test)[index,i].mean())
    
for i in range(8):
    print(lr.decision_function(X_Leptodactylidae_test)[index,i].mean())

0.9651544235
0.00293826614903
0.0189767251699
0.00489358591844
0.00396867509332
0.00226016066057
0.00135611631231
0.00045204719647
5.15975020443
-5.8348317298
-4.51733501912
-7.80463287103
-7.03525437898
-6.09788983447
-6.60959717103
-7.70908359773


In [98]:
np.unique(predicted_Bufonidae)

array(['Hypsiboas', 'Rhinella'], dtype=object)

In [103]:
np.unique(predicted_Leptodactylidae)

array(['Adenomera', 'Dendropsophus', 'Hypsiboas', 'Leptodactylus'], dtype=object)

In [104]:
np.unique(predicted_Dendrobatidae)

array(['Ameerega'], dtype=object)

In [110]:
predicted_Hylidae.shape

(2139,)

In [263]:
#matrix on Genus level
result_genus = np.c_[y_Hylidae, predicted_Hylidae]

Adenomera = 0
Ameerega = 0
Dendropsophus = 0
Hypsiboas = 0
Leptodactylus = 0
Osteocephalus = 0
Rhinella = 0
Scinax = 0
total = 0

for i in range(len(y_Hylidae)):
    if result_genus[:, 0][i] == 'Osteocephalus':
        total += 1
        if result_genus[:, 1][i] == 'Adenomera':
            Adenomera += 1
        if result_genus[:, 1][i] == 'Ameerega':
            Ameerega += 1
        if result_genus[:, 1][i] == 'Dendropsophus':
            Dendropsophus += 1
        if result_genus[:, 1][i] == 'Hypsiboas':
            Hypsiboas += 1
        if result_genus[:, 1][i] == 'Leptodactylus':
            Leptodactylus += 1
        if result_genus[:, 1][i] == 'Osteocephalus':
            Osteocephalus += 1
        if result_genus[:, 1][i] == 'Rhinella':
            Rhinella += 1
        if result_genus[:, 1][i] == 'Scinax':
            Scinax += 1

Adenomera, Ameerega, Dendropsophus, Hypsiboas, Leptodactylus, Osteocephalus, Rhinella, Scinax, total

(0, 0, 0, 91, 1, 8, 0, 1, 101)

In [155]:
#To copy the data predicted to be Adenomera
X_Adenomera = list()
y_Adenomera = list()
for i in range(len(predicted_Leptodactylidae)):
    if predicted_Leptodactylidae[i] == 'Adenomera':
        X_Adenomera.append(X_Leptodactylidae[i])
        
X_Adenomera = np.array(X_Adenomera)
y_Adenomera = np.array(X_Adenomera[:,24])

#To copy the data predicted to be Leptodactylus
X_Leptodactylus = list()
y_Leptodactylus = list()
for i in range(len(predicted_Leptodactylidae)):
    if predicted_Leptodactylidae[i] == 'Leptodactylus':
        X_Leptodactylus.append(X_Leptodactylidae[i])
        
X_Leptodactylus = np.array(X_Leptodactylus)
y_Leptodactylus = np.array(X_Leptodactylus[:,24])

#To copy the data predicted to be Scinax 
X_Scinax  = list()
y_Scinax  = list()
for i in range(len(predicted_Hylidae )):
    if predicted_Hylidae[i] == 'Scinax':
        X_Scinax.append(X_Hylidae[i])

        
X_Scinax = np.array(X_Scinax)
y_Scinax = np.array(X_Scinax[:,24])


#To copy the data predicted to be Hypsiboas
X_Hypsiboas = list()
y_Hypsiboas = list()
for i in range(len(predicted_Hylidae)):
    if predicted_Hylidae[i] == 'Hypsiboas':
        X_Hypsiboas.append(X_Hylidae[i])
        
X_Hypsiboas = np.array(X_Hypsiboas)
y_Hypsiboas = np.array(X_Hypsiboas[:,24])


#To copy the data predicted to be Osteocephalus
X_Osteocephalus = list()
y_Osteocephalus = list()
for i in range(len(predicted_Hylidae)):
    if predicted_Hylidae[i] == 'Osteocephalus':
        X_Osteocephalus.append(X_Hylidae[i])
        
X_Osteocephalus = np.array(X_Osteocephalus)
y_Osteocephalus = np.array(X_Osteocephalus[:,24])


#To copy the data predicted to be Dendropsophus
X_Dendropsophus = list()
y_Dendropsophus = list()
for i in range(len(predicted_Hylidae)):
    if predicted_Hylidae[i] == 'Dendropsophus':
        X_Dendropsophus.append(X_Hylidae[i])
        
X_Dendropsophus = np.array(X_Dendropsophus)
y_Dendropsophus = np.array(X_Dendropsophus[:,24])


#To copy the data predicted to be Rhinella
X_Rhinella = list()
y_Rhinella = list()
for i in range(len(predicted_Bufonidae  )):
    if predicted_Bufonidae  [i] == 'Rhinella':
        X_Rhinella.append(X_Bufonidae[i])
        
X_Rhinella = np.array(X_Rhinella)
y_Rhinella = np.array(X_Rhinella[:,24])


#To copy the data predicted to be Ameerega
X_Ameerega = list()
y_Ameerega = list()
for i in range(len(predicted_Dendrobatidae)):
    if predicted_Dendrobatidae[i] == 'Ameerega':
        X_Ameerega.append(X_Dendrobatidae[i])
        
X_Ameerega = np.array(X_Ameerega)
y_Ameerega = np.array(X_Ameerega[:,24])

In [422]:
#data prepare
print(np.unique(X_Ameerega[:,24]))
X_Ameerega_test = X_Ameerega[:,:22]

#group prepare
print(np.unique(X_Ameerega[:,25]).size)
groups_Ameerega = np.array(X_Ameerega[:,25])

['AdenomeraAndre' 'AdenomeraHylaedactylus' 'Ameeregatrivittata'
 'HylaMinuta' 'LeptodactylusFuscus' 'OsteocephalusOophagus']
20


In [423]:
#crossvalidation
gkf = GroupKFold(n_splits=20)
gkf.split(X_Ameerega_test, 
          y_Ameerega, 
          groups_Ameerega)

#LR on anural
lr = LogisticRegressionCV(cv=list(gkf.split(X_Ameerega_test, 
                                            y_Ameerega, 
                                            groups_Ameerega)))


lr.fit(X_Ameerega_test, y_Ameerega)

predicted_Ameerega = lr.predict(X_Ameerega_test)

for i in range(6):
    print(lr.predict_proba(X_Ameerega_test)[:,i].mean())

0.0663080850811
0.00179224370637
0.890678998626
0.0376343201183
0.00179452455453
0.00179182791369


In [425]:
index = list()
for i in range(len(predicted_Ameerega)):
    if predicted_Ameerega[i] == 'Ameeregatrivittata':
        index.append(i)
                
for i in range(6):
    print(lr.predict_proba(X_Ameerega_test)[index,i].mean())
    
for i in range(6):
    print(lr.decision_function(X_Ameerega_test)[index,i].mean())

0.0663080850811
0.00179224370637
0.890678998626
0.0376343201183
0.00179452455453
0.00179182791369
-2.64483220379
-6.32249077163
2.0977171029
-3.24147566776
-6.32121667484
-6.32272321047


In [418]:
#data prepare
print(np.unique(X_Rhinella[:,24]))
X_Rhinella_test = X_Rhinella[:,:22]

#group prepare
print(np.unique(X_Rhinella[:,25]).size)
groups_Rhinella = np.array(X_Rhinella[:,25])

['AdenomeraAndre' 'Ameeregatrivittata' 'HylaMinuta' 'LeptodactylusFuscus'
 'OsteocephalusOophagus' 'Rhinellagranulosa' 'ScinaxRuber']
8


In [419]:
#crossvalidation
gkf = GroupKFold(n_splits=8)
gkf.split(X_Rhinella_test, 
          y_Rhinella, 
          groups_Rhinella)

#LR on anural
lr = LogisticRegressionCV(cv=list(gkf.split(X_Rhinella_test, 
                                            y_Rhinella, 
                                            groups_Rhinella)))


lr.fit(X_Rhinella_test, y_Rhinella)

predicted_Rhinella = lr.predict(X_Rhinella_test)

for i in range(7):
    print(lr.predict_proba(X_Rhinella_test)[:,i].mean())

0.0833333504481
0.0555554170213
0.0833333032105
0.0277777815311
0.0555556053993
0.611111192668
0.0833333497213


In [421]:
index = list()
for i in range(len(predicted_Rhinella)):
    if predicted_Rhinella[i] == 'Rhinellagranulosa':
        index.append(i)
                
for i in range(7):
    print(lr.predict_proba(X_Rhinella_test)[index,i].mean())
    
for i in range(7):
    print(lr.decision_function(X_Rhinella_test)[index,i].mean())

0.0833333504481
0.0555554170213
0.0833333032105
0.0277777815311
0.0555556053993
0.611111192668
0.0833333497213
-2.39789520816
-2.83321613889
-2.39789583176
-3.55534807159
-2.83321255389
0.451985094659
-2.39789521649


In [414]:
#data prepare
print(np.unique(X_Dendropsophus[:,24]))
X_Dendropsophus_test = X_Dendropsophus[:,:22]

#group prepare
print(np.unique(X_Dendropsophus[:,25]).size)
groups_Dendropsophus = np.array(X_Dendropsophus[:,25])

['AdenomeraAndre' 'AdenomeraHylaedactylus' 'Ameeregatrivittata'
 'HylaMinuta' 'LeptodactylusFuscus']
18


In [415]:
#crossvalidation
gkf = GroupKFold(n_splits=18)
gkf.split(X_Dendropsophus_test, 
          y_Dendropsophus, 
          groups_Dendropsophus)

#LR on anural
lr = LogisticRegressionCV(cv=list(gkf.split(X_Dendropsophus_test, 
                                            y_Dendropsophus, 
                                            groups_Dendropsophus)))


lr.fit(X_Dendropsophus_test, y_Dendropsophus)

predicted_Dendropsophus = lr.predict(X_Dendropsophus_test)

for i in range(5):
    print(lr.predict_proba(X_Dendropsophus_test)[:,i].mean())

0.0291584620965
0.00971964122355
0.082617003278
0.87364454693
0.00486034647235


In [417]:
index = list()
for i in range(len(predicted_Dendropsophus)):
    if predicted_Dendropsophus[i] == 'HylaMinuta':
        index.append(i)
                
for i in range(5):
    print(lr.predict_proba(X_Dendropsophus_test)[index,i].mean())
    
for i in range(5):
    print(lr.decision_function(X_Dendropsophus_test)[index,i].mean())

0.0291584620965
0.00971964122355
0.082617003278
0.87364454693
0.00486034647235
-3.55060949044
-4.66814559646
-2.45509332777
2.79923475615
-5.36586990247


In [409]:
#data prepare
print(np.unique(X_Osteocephalus[:,24]))
X_Osteocephalus_test = X_Osteocephalus[:,:22]

#group prepare
print(np.unique(X_Osteocephalus[:,25]).size)
groups_Osteocephalus = np.array(X_Osteocephalus[:,25])

['AdenomeraAndre' 'AdenomeraHylaedactylus' 'Ameeregatrivittata'
 'HypsiboasCinerascens' 'HypsiboasCordobae' 'OsteocephalusOophagus']
9


In [410]:
#crossvalidation
gkf = GroupKFold(n_splits=9)
gkf.split(X_Osteocephalus_test, 
          y_Osteocephalus, 
          groups_Osteocephalus)

#LR on anural
lr = LogisticRegressionCV(cv=list(gkf.split(X_Osteocephalus_test, 
                                            y_Osteocephalus, 
                                            groups_Osteocephalus)))


lr.fit(X_Osteocephalus_test, y_Osteocephalus)

predicted_Osteocephalus = lr.predict(X_Osteocephalus_test)

for i in range(6):
    print(lr.predict_proba(X_Osteocephalus_test)[:,i].mean())

0.196903015835
0.0492248983125
0.261598237684
0.0492259488032
0.0492256457713
0.393822253594


In [411]:
index = list()
for i in range(len(predicted_Osteocephalus)):
    if predicted_Osteocephalus[i] == 'OsteocephalusOophagus':
        index.append(i)

In [413]:
for i in range(6):
    print(lr.predict_proba(X_Osteocephalus_test)[index,i].mean())
    
for i in range(6):
    print(lr.decision_function(X_Osteocephalus_test)[index,i].mean())

0.256794641573
0.0641963773041
0.0369726830037
0.0641993579061
0.0641985053412
0.513638434872
-1.60940605974
-3.13550065869
-4.6209987651
-3.13545539221
-3.13546823286
-0.692967568147


In [401]:
#data prepare
print(np.unique(X_Hypsiboas[:,24]))
X_Hypsiboas_test = X_Hypsiboas[:,:22]

#group prepare
print(np.unique(X_Hypsiboas[:,25]).size)
groups_Hypsiboas = np.array(X_Hypsiboas[:,25])

['AdenomeraAndre' 'AdenomeraHylaedactylus' 'Ameeregatrivittata'
 'HylaMinuta' 'HypsiboasCinerascens' 'HypsiboasCordobae'
 'LeptodactylusFuscus' 'OsteocephalusOophagus' 'Rhinellagranulosa'
 'ScinaxRuber']
34


In [402]:
#crossvalidation
gkf = GroupKFold(n_splits=34)
gkf.split(X_Hypsiboas_test, 
          y_Hypsiboas, 
          groups_Hypsiboas)

#LR on anural
lr = LogisticRegressionCV(cv=list(gkf.split(X_Hypsiboas_test, 
                                            y_Hypsiboas, 
                                            groups_Hypsiboas)))


lr.fit(X_Hypsiboas_test, y_Hypsiboas)

predicted_Hypsiboas = lr.predict(X_Hypsiboas_test)

for i in range(10):
    print(lr.predict_proba(X_Hypsiboas_test)[:,i].mean())

0.0167964453036
0.00183495783437
0.00129880854574
0.00218196962474
0.29088185741
0.595547970816
0.00844216386317
0.0591044768763
0.0206644425979
0.00324690712893


In [406]:
index = list()
for i in range(len(predicted_Hypsiboas)):
    if predicted_Hypsiboas[i] == 'HypsiboasCinerascens':
        index.append(i)

In [407]:
for i in range(10):
    print(lr.predict_proba(X_Hypsiboas_test)[index,i].mean())

0.00987302990911
0.000213947669045
0.00158085505218
1.62992746966e-05
0.877888293652
0.0216545585863
0.0102763861655
0.071978344315
0.00256617391142
0.00395211146494


In [408]:
for i in range(10):
    print(lr.decision_function(X_Hypsiboas_test)[index,i].mean())

-7.0992674044
-14.9118572152
-6.75925405742
-18.2335652025
1.85716296004
-8.4622061603
-4.88098982846
-2.88795251673
-14.4199414111
-5.84122050729


In [396]:
#data prepare
print(np.unique(X_Scinax[:,24]))
X_Scinax_test = X_Scinax[:,:22]

#group prepare
print(np.unique(X_Scinax[:,25]).size)
groups_Scinax = np.array(X_Scinax[:,25])

['AdenomeraAndre' 'AdenomeraHylaedactylus' 'Ameeregatrivittata'
 'HypsiboasCinerascens' 'HypsiboasCordobae' 'OsteocephalusOophagus'
 'ScinaxRuber']
12


In [397]:
#crossvalidation
gkf = GroupKFold(n_splits=12)
gkf.split(X_Scinax_test, 
          y_Scinax, 
          groups_Scinax)

#LR on anural
lr = LogisticRegressionCV(cv=list(gkf.split(X_Scinax_test, 
                                            y_Scinax, 
                                            groups_Scinax)))


lr.fit(X_Scinax_test, y_Scinax)

predicted_Scinax = lr.predict(X_Scinax_test)

for i in range(7):
    print(lr.predict_proba(X_Scinax_test)[:,i].mean())

0.0115899872506
0.00686396535926
0.0137279203961
0.00686395517827
0.00686396315625
0.00686385125109
0.947226357408


In [398]:
index = list()
for i in range(len(predicted_Scinax)):
    if predicted_Scinax[i] == 'ScinaxRuber':
        index.append(i)

In [399]:
for i in range(7):
    print(lr.predict_proba(X_Scinax_test)[index,i].mean())

0.00823421346626
0.00688726996292
0.0137745280511
0.0068872586473
0.00688726651052
0.00688715399633
0.950442309366


In [400]:
for i in range(7):
    print(lr.decision_function(X_Scinax_test)[index,i].mean())

-9.14509636592
-4.98360591065
-4.28358711288
-4.98360663099
-4.98360643509
-4.98362239073
2.73002296704


In [388]:
#data prepare
print(np.unique(X_Adenomera[:,24]))
X_Adenomera_test = X_Adenomera[:,:22]

#group prepare
print(np.unique(X_Adenomera[:,25]).size)
groups_Adenomera = np.array(X_Adenomera[:,25])

['AdenomeraAndre' 'AdenomeraHylaedactylus' 'Ameeregatrivittata'
 'HylaMinuta' 'HypsiboasCinerascens' 'HypsiboasCordobae'
 'LeptodactylusFuscus' 'OsteocephalusOophagus' 'Rhinellagranulosa'
 'ScinaxRuber']
42


In [389]:
#crossvalidation
gkf = GroupKFold(n_splits=42)
gkf.split(X_Adenomera_test, 
          y_Adenomera, 
          groups_Adenomera)

#LR on anural
lr = LogisticRegressionCV(cv=list(gkf.split(X_Adenomera_test, 
                                            y_Adenomera, 
                                            groups_Adenomera)))


lr.fit(X_Adenomera_test, y_Adenomera)

predicted_Adenomera = lr.predict(X_Adenomera_test)

for i in range(9):
    print(lr.predict_proba(X_Adenomera_test)[:,i].mean())

0.143424832965
0.815313880708
0.00362232417081
0.0227305802777
0.00197566328961
0.00733503186625
0.00131707723045
0.00263424926232
0.0013171031904


In [393]:
index = list()
for i in range(len(predicted_Adenomera)):
    if predicted_Adenomera[i] == 'AdenomeraHylaedactylus':
        index.append(i)

In [394]:
for i in range(9):
    print(lr.predict_proba(X_Adenomera_test)[index,i].mean())

1.04834894736e-07
0.972707062311
0.00271914356332
0.0170505090016
0.00148318029916
0.00183760488571
0.000988816629385
0.00197752552828
0.000988831344609


In [395]:
for i in range(8):
    print(lr.decision_function(X_Adenomera_test)[index,i].mean())

-36.4693424448
9.6767440675
-5.93596519282
-4.08612101335
-6.54328476045
-7.78950291292
-6.94920049475
-6.255148468


In [173]:
#data prepare
print(np.unique(X_Leptodactylus[:,24]))
X_Leptodactylus_test = X_Leptodactylus[:,:22]

#group prepare
print(np.unique(X_Leptodactylus[:,25]).size)
groups_Leptodactylus = np.array(X_Leptodactylus[:,25])

['AdenomeraAndre' 'AdenomeraHylaedactylus' 'Ameeregatrivittata'
 'HylaMinuta' 'HypsiboasCinerascens' 'HypsiboasCordobae'
 'LeptodactylusFuscus' 'OsteocephalusOophagus' 'Rhinellagranulosa']
12


In [383]:
#crossvalidation
gkf = GroupKFold(n_splits=12)
gkf.split(X_Leptodactylus_test, 
          y_Leptodactylus, 
          groups_Leptodactylus)

#LR on anural
lr = LogisticRegressionCV(cv=list(gkf.split(X_Leptodactylus_test, 
                                            y_Leptodactylus, 
                                            groups_Leptodactylus)))


lr.fit(X_Leptodactylus_test, y_Leptodactylus)

predicted_Leptodactylus = lr.predict(X_Leptodactylus_test)

for i in range(9):
    print(lr.predict_proba(X_Leptodactylus_test)[:,i].mean())

0.00449739121708
0.00449741905923
0.00449726306393
0.00899485146853
0.00899478223901
0.00449739182625
0.950528702282
0.00449739170189
0.00899480714175


In [384]:
X_Leptodactylus_test.shape

(235, 22)

In [385]:
index = list()
for i in range(len(predicted_Leptodactylus)):
    if predicted_Leptodactylus[i] == 'LeptodactylusFuscus':
        index.append(i)

In [386]:
for i in range(9):
    print(lr.predict_proba(X_Leptodactylus_test)[index,i].mean())

0.00449739121708
0.00449741905923
0.00449726306393
0.00899485146853
0.00899478223901
0.00449739182625
0.950528702282
0.00449739170189
0.00899480714175


In [387]:
for i in range(9):
    print(lr.decision_function(X_Leptodactylus_test)[index,i].mean())

-5.4553208662
-5.45532020239
-5.45535343981
-4.75789097209
-4.75789062061
-5.45532027625
4.1724775024
-5.45532073609
-4.75788846191


In [262]:
#matrix on Genus level
result_species = np.c_[y_Ameerega, predicted_Ameerega]

AdenomeraAndre = 0
AdenomeraHylaedactylus = 0
Ameeregatrivittata = 0
HylaMinuta = 0
HypsiboasCinerascens = 0
HypsiboasCordobae = 0
LeptodactylusFuscus = 0
OsteocephalusOophagus = 0
Rhinellagranulosa = 0
ScinaxRuber = 0
total = 0

for i in range(len(y_Ameerega)):
    if result_species[:, 0][i] == 'ScinaxRuber':
        total += 1
        if result_species[:, 1][i] == 'AdenomeraAndre':
            AdenomeraAndre += 1
        if result_species[:, 1][i] == 'AdenomeraHylaedactylus':
            AdenomeraHylaedactylus += 1
        if result_species[:, 1][i] == 'Ameeregatrivittata':
            Ameeregatrivittata += 1
        if result_species[:, 1][i] == 'HylaMinuta':
            HylaMinuta += 1
        if result_species[:, 1][i] == 'HypsiboasCinerascens':
            HypsiboasCinerascens += 1
        if result_species[:, 1][i] == 'HypsiboasCordobae':
            HypsiboasCordobae += 1
        if result_species[:, 1][i] == 'LeptodactylusFuscus':
            LeptodactylusFuscus += 1
        if result_species[:, 1][i] == 'OsteocephalusOophagus':
            OsteocephalusOophagus += 1
        if result_species[:, 1][i] == 'Rhinellagranulosa':
            Rhinellagranulosa += 1
        if result_species[:, 1][i] == 'ScinaxRuber':
            ScinaxRuber += 1

AdenomeraAndre,AdenomeraHylaedactylus,Ameeregatrivittata,HylaMinuta,HypsiboasCinerascens,HypsiboasCordobae,LeptodactylusFuscus,OsteocephalusOophagus,Rhinellagranulosa,ScinaxRuber,total

(0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0)

0.5882352941176471